In [16]:
import pandas as pd
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, XLNetTokenizer, XLNetForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import seaborn as sns

# Load the preprocessed dataset
df_reviews = pd.read_csv("../Data/processed/df_reviews.csv")


In [17]:
df_reviews = df_reviews.tail(5000)

In [18]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(df_reviews['text'], df_reviews['sentiment'], test_size=0.2, random_state=42)


In [19]:
# Tokenization for RoBERTa
tokenizer_roberta = RobertaTokenizer.from_pretrained('roberta-base')
train_encodings_roberta = tokenizer_roberta(X_train.tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt")
test_encodings_roberta = tokenizer_roberta(X_test.tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt")


In [20]:
# Tokenization for XLNet
tokenizer_xlnet = XLNetTokenizer.from_pretrained('xlnet-base-cased')
train_encodings_xlnet = tokenizer_xlnet(X_train.tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt")
test_encodings_xlnet = tokenizer_xlnet(X_test.tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt")

In [21]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [22]:
# Convert labels to numerical values
label_map = {'negative': 0, 'neutral': 1, 'positive': 2}
y_train_numeric = y_train.map(label_map).tolist()
y_test_numeric = y_test.map(label_map).tolist()


In [23]:
# Create datasets
train_dataset_roberta = CustomDataset(train_encodings_roberta, y_train_numeric)
test_dataset_roberta = CustomDataset(test_encodings_roberta, y_test_numeric)

In [24]:
train_dataset_xlnet = CustomDataset(train_encodings_xlnet, y_train_numeric)
test_dataset_xlnet = CustomDataset(test_encodings_xlnet, y_test_numeric)

In [25]:
# Define the RoBERTa model
model_roberta = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
# Define the XLNet model
model_xlnet = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=3)


Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

c:\Users\abdel\anaconda3\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
# Trainer for RoBERTa
trainer_roberta = Trainer(
    model=model_roberta,
    args=training_args,
    train_dataset=train_dataset_roberta,
    eval_dataset=test_dataset_roberta,
)

In [29]:
# Trainer for XLNet
trainer_xlnet = Trainer(
    model=model_xlnet,
    args=training_args,
    train_dataset=train_dataset_xlnet,
    eval_dataset=test_dataset_xlnet,
)

In [30]:
# Train the models
trainer_roberta.train()

  0%|          | 0/750 [00:00<?, ?it/s]

{'loss': 1.13, 'grad_norm': 3.7246458530426025, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.04}
{'loss': 1.127, 'grad_norm': 4.371917724609375, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.08}
{'loss': 1.104, 'grad_norm': 3.795968532562256, 'learning_rate': 3e-06, 'epoch': 0.12}
{'loss': 1.0625, 'grad_norm': 4.1021037101745605, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.16}
{'loss': 0.9453, 'grad_norm': 3.269951105117798, 'learning_rate': 5e-06, 'epoch': 0.2}
{'loss': 0.8366, 'grad_norm': 6.965307712554932, 'learning_rate': 6e-06, 'epoch': 0.24}
{'loss': 0.7191, 'grad_norm': 4.5477399826049805, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.28}
{'loss': 0.6046, 'grad_norm': 8.081356048583984, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.32}
{'loss': 0.7518, 'grad_norm': 18.291521072387695, 'learning_rate': 9e-06, 'epoch': 0.36}
{'loss': 0.6408, 'grad_norm': 12.932360649108887, 'learning_rate': 1e-05, 'epoch': 0.4}
{'loss': 0.5482, 'grad_norm': 6.9610447

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.5057488083839417, 'eval_runtime': 124.0961, 'eval_samples_per_second': 8.058, 'eval_steps_per_second': 0.508, 'epoch': 1.0}
{'loss': 0.4619, 'grad_norm': 10.760321617126465, 'learning_rate': 2.6000000000000002e-05, 'epoch': 1.04}
{'loss': 0.4254, 'grad_norm': 4.914262294769287, 'learning_rate': 2.7000000000000002e-05, 'epoch': 1.08}
{'loss': 0.5158, 'grad_norm': 21.053640365600586, 'learning_rate': 2.8000000000000003e-05, 'epoch': 1.12}
{'loss': 0.5562, 'grad_norm': 15.478263854980469, 'learning_rate': 2.9e-05, 'epoch': 1.16}
{'loss': 0.373, 'grad_norm': 6.173689842224121, 'learning_rate': 3e-05, 'epoch': 1.2}
{'loss': 0.3819, 'grad_norm': 23.08915138244629, 'learning_rate': 3.1e-05, 'epoch': 1.24}
{'loss': 0.378, 'grad_norm': 15.639979362487793, 'learning_rate': 3.2000000000000005e-05, 'epoch': 1.28}
{'loss': 0.37, 'grad_norm': 21.638898849487305, 'learning_rate': 3.3e-05, 'epoch': 1.32}
{'loss': 0.5545, 'grad_norm': 9.286733627319336, 'learning_rate': 3.40000000000000

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.5603524446487427, 'eval_runtime': 149.6435, 'eval_samples_per_second': 6.683, 'eval_steps_per_second': 0.421, 'epoch': 2.0}
{'loss': 0.4884, 'grad_norm': 1.807286024093628, 'learning_rate': 4.8e-05, 'epoch': 2.04}
{'loss': 0.6364, 'grad_norm': 9.66075325012207, 'learning_rate': 4.600000000000001e-05, 'epoch': 2.08}
{'loss': 0.5098, 'grad_norm': 4.2256011962890625, 'learning_rate': 4.4000000000000006e-05, 'epoch': 2.12}
{'loss': 0.5304, 'grad_norm': 8.858053207397461, 'learning_rate': 4.2e-05, 'epoch': 2.16}
{'loss': 0.4711, 'grad_norm': 43.19972610473633, 'learning_rate': 4e-05, 'epoch': 2.2}
{'loss': 0.414, 'grad_norm': 18.895606994628906, 'learning_rate': 3.8e-05, 'epoch': 2.24}
{'loss': 0.555, 'grad_norm': 11.44795036315918, 'learning_rate': 3.6e-05, 'epoch': 2.28}
{'loss': 0.4163, 'grad_norm': 18.696142196655273, 'learning_rate': 3.4000000000000007e-05, 'epoch': 2.32}
{'loss': 0.3911, 'grad_norm': 4.72122859954834, 'learning_rate': 3.2000000000000005e-05, 'epoch': 2

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.38916173577308655, 'eval_runtime': 175.7527, 'eval_samples_per_second': 5.69, 'eval_steps_per_second': 0.358, 'epoch': 3.0}
{'train_runtime': 31385.4272, 'train_samples_per_second': 0.382, 'train_steps_per_second': 0.024, 'train_loss': 0.5389129981994629, 'epoch': 3.0}


TrainOutput(global_step=750, training_loss=0.5389129981994629, metrics={'train_runtime': 31385.4272, 'train_samples_per_second': 0.382, 'train_steps_per_second': 0.024, 'total_flos': 789340253184000.0, 'train_loss': 0.5389129981994629, 'epoch': 3.0})

In [ ]:
trainer_xlnet.train()

  0%|          | 0/750 [00:00<?, ?it/s]

{'loss': 1.0539, 'grad_norm': 32.7119026184082, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.04}
{'loss': 0.9647, 'grad_norm': 17.651409149169922, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.08}
{'loss': 0.9046, 'grad_norm': 17.554433822631836, 'learning_rate': 3e-06, 'epoch': 0.12}
{'loss': 0.8885, 'grad_norm': 24.23990821838379, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.16}
{'loss': 0.7815, 'grad_norm': 14.04226016998291, 'learning_rate': 5e-06, 'epoch': 0.2}
{'loss': 0.8017, 'grad_norm': 19.474647521972656, 'learning_rate': 6e-06, 'epoch': 0.24}
{'loss': 0.7377, 'grad_norm': 13.498127937316895, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.28}


In [ ]:
# Evaluate RoBERTa
predictions_roberta = trainer_roberta.predict(test_dataset_roberta)
y_pred_roberta = np.argmax(predictions_roberta.predictions, axis=1)


In [ ]:
# Evaluate XLNet
predictions_xlnet = trainer_xlnet.predict(test_dataset_xlnet)
y_pred_xlnet = np.argmax(predictions_xlnet.predictions, axis=1)

In [ ]:
# Save the predictions
np.save('roberta_predictions.npy', y_pred_roberta)
np.save('xlnet_predictions.npy', y_pred_xlnet)

In [ ]:
# Plot training loss
def plot_training_loss(trainer, model_name):
    plt.figure(figsize=(10, 6))
    plt.plot(trainer.state.log_history, label='Training Loss')
    plt.title(f'{model_name} Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

plot_training_loss(trainer_roberta, 'RoBERTa')
plot_training_loss(trainer_xlnet, 'XLNet')


In [ ]:
# Plot evaluation metrics
def plot_evaluation_metrics(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    metrics = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'Score': [accuracy, precision, recall, f1]
    })

    plt.figure(figsize=(10, 6))
    sns.barplot(data=metrics, x='Metric', y='Score')
    plt.title(f'{model_name} Evaluation Metrics')
    plt.xlabel('Metric')
    plt.ylabel('Score')
    plt.show()

plot_evaluation_metrics(y_test_numeric, y_pred_roberta, 'RoBERTa')
plot_evaluation_metrics(y_test_numeric, y_pred_xlnet, 'XLNet')